In [55]:
# set dependencies
import os
import csv
import requests
import time
import psycopg2
import pymongo
import json
import pprint
import urllib.request

import pandas as pd
import numpy as np

from flask_pymongo import PyMongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
from api_keys import nps_key

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows=100

# National Parks Visitor table

In [40]:
# Get data table
tables=pd.read_html('https://www.nps.gov/aboutus/visitation-numbers.htm', index_col=0)
table=pd.DataFrame(tables[1])
table

,Park,Recreational Visits
1,Great Smoky Mountains National Park,12.1 million
2,Yellowstone National Park,"3,8 million"
3,Zion National Park,3.6 million
4,Rocky Mountain National Park,3.3 million
5,Grand Teton National Park,3.3 million
6,Grand Canyon National Park,2.9 million
7,Cuyahoga Valley National Park,2.8 million
8,Acadia National Park,2.7 million
9,Olympic National Park,2.5 million
10,Joshua Tree National Park,2.4 million


# Kaggle National Parks

In [41]:
parks=pd.read_csv('dataFiles/parks.csv')
parks.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


# Kaggle National Parks species

In [42]:
species=pd.read_csv('dataFiles/species.csv',index_col=0,skipinitialspace=True,usecols=[1,2,3,4,5,6,7,8,9,10,11,12])
species.head()

,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status
Park Name,,,,,,,,,,,
Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN
Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN
Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern
Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered
Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN


# National Park Service API set-up

In [51]:
base_url='https://developer.nps.gov/api/v1/'

# API routes for API looping
nps_api_list=['/activities','/activities/parks','/alerts','/amenities','/amenities/parksplaces',
              '/amenities/parksvisitorcenters','/articles','/campgrounds','/events','/lessonplans','/newsreleases','/parks',
              '/passportstamplocations','/people','/places','/thingstodo','/topics','/topics/parks','/tours','/visitorcenters',
              '/webcams']

# Name list for API looping
nps_api_names=['activities','activitiesparks','alerts','amenities','amenitiesparksplaces','amenitiesparksvisitorcenters',
               'articles','campgrounds','events','lessonplans','newsreleases','parks','passportstamplocations','people',
               'places','thingstodo','topics','topicsparks','tours','visitorcenters','webcams']

# Merge Table and parks dataframes and create code list for API looping
table_merged=table.merge(parks,'left',left_on='Park',right_on='Park Name')
nps_park_codes= parks['Park Code'].tolist()
top_ten_park_codes=table_merged['Park Code'].tolist()
top_ten_park_codes=[x.lower() for x in top_ten_park_codes]
pprint.pformat(top_ten_park_codes,compact=True)



"['grsm', 'yell', 'zion', 'romo', 'grte', 'grca', 'cuva', 'acad', 'olym', 'jotr']"

In [81]:
# Configure API request
endpoint = f"https://developer.nps.gov/api/v1/parks?parkCode=grsm,yell,zion,romo,grte,grca,cuva,acad,olym,jotr"
HEADERS = {"Authorization":nps_key}
req = urllib.request.Request(endpoint,headers=HEADERS)
req